In [2]:
import os
%pwd

'/Users/karandeepsingh/Desktop/MLOps/TextSummarizer/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/karandeepsingh/Desktop/MLOps/TextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.commons import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=int(params.save_steps),
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay
        )

        return model_trainer_config

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/Users/karandeepsingh/Desktop/MLOps/TextSummarizer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config=config
    
    def train(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading the data
        dataset_samsum_pt=load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            per_device_eval_batch_size=1
        )
        
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()
        
        ## Save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegaus-samsum-model"))
        
        ## Save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        

In [10]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
  Using cached transformers-4.56.1-py3-none-any.whl.metadata (42 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.56.1-py3-none-any.whl (11.6 MB)
Using cached accelerate-1.10.1-py3-none-any.whl (374 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [13]:
config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer_config.data_path

[2025-09-11 17:15:55,769: INFO: commons: yaml file: config/config.yaml loaded successfully]
[2025-09-11 17:15:55,772: INFO: commons: yaml file: params.yaml loaded successfully]
[2025-09-11 17:15:55,773: INFO: commons: created a directory at: artifacts]
[2025-09-11 17:15:55,774: INFO: commons: created a directory at: artifacts/model_trainer]


'artifacts/data_transformation/samsum_dataset'

In [21]:
from datasets import load_from_disk

dataset = load_from_disk("artifacts/data_transformation/samsum_dataset")
df = dataset['train'].to_pandas()
df.head()


,id,dialogue,summary,input_ids,attention_mask,labels
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...,"[12195, 151, 125, 7091, 3659, 107, 842, 119, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12195, 7091, 3659, 111, 138, 650, 10508, 181,..."
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...,"[18038, 151, 2632, 127, 119, 6228, 118, 115, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[18038, 111, 34296, 127, 6228, 118, 33195, 115..."
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...,"[4776, 151, 4451, 108, 180, 131, 116, 164, 152...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5377, 218, 508, 109, 891, 93882, 3111, 2087, ..."
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...,"[7535, 151, 9199, 108, 125, 311, 125, 131, 208...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[7535, 7234, 178, 117, 115, 298, 122, 14521, 1..."
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com...","[4037, 151, 14381, 61237, 110, 13137, 416, 364...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4037, 117, 6436, 108, 262, 178, 61237, 7888, ..."


In [22]:
config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer=ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-09-11 17:24:43,081: INFO: commons: yaml file: config/config.yaml loaded successfully]
[2025-09-11 17:24:43,088: INFO: commons: yaml file: params.yaml loaded successfully]
[2025-09-11 17:24:43,091: INFO: commons: created a directory at: artifacts]
[2025-09-11 17:24:43,093: INFO: commons: created a directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/fp/5wj5hfvn6x9fd8jmprqz_4fm0000gn/T/ipykernel_5391/1387716570.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/Users/karandeepsingh/Desktop/MLOps/TextSummarizer/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning

RuntimeError: MPS backend out of memory (MPS allocated: 7.34 GiB, other allocations: 1.48 GiB, max allowed: 9.07 GiB). Tried to allocate 375.40 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
import accelerate
print(accelerate.__version__)

1.10.1
